In [ ]:
!pip -q install ucimlrepo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.2 MB/s eta 0:00:00


In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
adult = fetch_ucirepo(id=2)

def load_data():
    X = adult.data.features
    y = adult.data.targets
    for col in X.select_dtypes(include="object").columns:
      X[col] = pd.factorize(X[col])[0]  # Encode categorical variables as integers
    for col in y.select_dtypes(include="object").columns:
      y[col] = pd.factorize(y[col])[0]  # Encode categorical variables as integers
    return X, y


In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict, deque

# Hoeffding Tree with Gini Index

In [ ]:
class HoeffdingTreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, is_leaf=True):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.is_leaf = is_leaf
        self.class_counts = defaultdict(int)
        self.data_points = []  # Store data points until a split


In [ ]:
class HoeffdingTree:
    def __init__(self, delta=0.01):
        self.root = HoeffdingTreeNode()
        self.delta = delta
        self.n = 0  # Number of samples seen

    def hoeffding_bound(self, R, n):
        return np.sqrt((R**2 * np.log(1 / self.delta)) / (2 * n))

    def update(self, x, y):
        self.n += 1
        node = self.root
        while not node.is_leaf:
            if x[node.feature] < node.threshold:
                node = node.left
            else:
                node = node.right

        node.class_counts[y] += 1
        node.data_points.append((x, y))

        if self.n % 100 == 0:  # Check for split every 100 samples
            self.attempt_split(node)

    def attempt_split(self, node):
        best_feature, best_threshold = None, None
        best_gini = float('inf')

        # Loop over features
        for feature in range(len(node.data_points[0][0])):  # Number of features
            feature_values = [x[feature] for x, _ in node.data_points]
            thresholds = np.unique(feature_values)

            for threshold in thresholds:
                left_counts = defaultdict(int)
                right_counts = defaultdict(int)

                for x, y in node.data_points:
                    if x[feature] < threshold:
                        left_counts[y] += 1
                    else:
                        right_counts[y] += 1

                gini = self.calculate_gini(left_counts, right_counts)
                if gini < best_gini:
                    best_gini = gini
                    best_feature = feature
                    best_threshold = threshold

        # Perform split if a suitable feature and threshold were found
        if best_feature is not None:
            node.feature = best_feature
            node.threshold = best_threshold
            node.is_leaf = False
            node.left = HoeffdingTreeNode()
            node.right = HoeffdingTreeNode()

            # Distribute data points to child nodes
            for x, y in node.data_points:
                if x[best_feature] < best_threshold:
                    node.left.class_counts[y] += 1
                    node.left.data_points.append((x, y))
                else:
                    node.right.class_counts[y] += 1
                    node.right.data_points.append((x, y))

            node.data_points = []  # Clear data points after splitting

    def calculate_gini(self, left_counts, right_counts):
        total_left = sum(left_counts.values())
        total_right = sum(right_counts.values())
        total = total_left + total_right
        gini_left = 1.0 - sum((count / total_left) ** 2 for count in left_counts.values()) if total_left > 0 else 0
        gini_right = 1.0 - sum((count / total_right) ** 2 for count in right_counts.values()) if total_right > 0 else 0
        return (total_left / total) * gini_left + (total_right / total) * gini_right

    def predict(self, x):
        node = self.root
        while not node.is_leaf:
            if x[node.feature] < node.threshold:
                node = node.left
            else:
                node = node.right

        # If node.class_counts is empty, return a default class (e.g., majority class in the dataset)
        if node.class_counts:
            return max(node.class_counts, key=node.class_counts.get)
        else:
            # Default class if no counts in this leaf (returning the most frequent class seen globally)
            return max(self.root.class_counts, key=self.root.class_counts.get)



In [ ]:
from sklearn.model_selection import train_test_split

X, y = load_data()  # Replace with actual data loading function
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
df = pd.concat([X_train, y_train], axis=1)

<ipython-input-2-6d554f8915d0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = pd.factorize(X[col])[0]  # Encode categorical variables as integers
<ipython-input-2-6d554f8915d0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = pd.factorize(X[col])[0]  # Encode categorical variables as integers
<ipython-input-2-6d554f8915d0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [ ]:
tree = HoeffdingTree()
for index, row in df.iterrows():
    x1 = row[:-1].values  # Features
    y1 = row.iloc[-1]     # Label
    tree.update(x1, y1)

In [ ]:
# Predict on new data
y_pred = []
for index, row in X_test.iterrows():
    x1 = row.values  # Features
    prediction = tree.predict(x1)
    y_pred.append(prediction)
    # print(f'Prediction: {prediction}')


In [ ]:
y_act = y_test.values.flatten()

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

# Assuming y_pred and y_act are your predicted and actual labels
accuracy = accuracy_score(y_act, y_pred)
f1 = f1_score(y_act, y_pred, average='macro')  # Use 'micro', 'macro', or 'weighted' based on your case
precision = precision_score(y_act, y_pred, average='macro')
recall = recall_score(y_act, y_pred, average='macro')
cm = confusion_matrix(y_act, y_pred)

# Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score (Macro): {f1:.4f}")
print(f"Precision (Macro): {precision:.4f}")
print(f"Recall (Macro): {recall:.4f}")
print(f"Confusion Matrix:\n{cm}")



Accuracy: 0.5784
F1 Score (Macro): 0.3690
Precision (Macro): 0.4928
Recall (Macro): 0.4054
Confusion Matrix:
[[4465  328  125   18]
 [ 591  929    7   35]
 [2063  171  234   10]
 [ 303  455   13   22]]


# Hoeffding Tree with only Information Gain

In [ ]:
import numpy as np
from collections import defaultdict

class HoeffdingTreeNode:
    def __init__(self):
        self.is_leaf = True
        self.feature = None
        self.threshold = None
        self.left = None
        self.right = None
        self.class_counts = defaultdict(int)
        self.data_points = []

class HoeffdingTree:
    def __init__(self, delta=0.01):
        self.root = HoeffdingTreeNode()
        self.delta = delta
        self.n = 0  # Number of samples seen

    def hoeffding_bound(self, R, n):
        return np.sqrt((R**2 * np.log(1 / self.delta)) / (2 * n))

    def update(self, x, y):
        self.n += 1
        node = self.root
        while not node.is_leaf:
            if x[node.feature] < node.threshold:
                node = node.left
            else:
                node = node.right

        node.class_counts[y] += 1
        node.data_points.append((x, y))

        if self.n % 100 == 0:  # Check for split every 100 samples
            self.attempt_split(node)

    def calculate_entropy(self, class_counts):
        total = sum(class_counts.values())
        return -sum((count / total) * np.log2(count / total) for count in class_counts.values() if count > 0)

    def calculate_information_gain(self, left_counts, right_counts):
        # Total counts before split
        total_counts = defaultdict(int, left_counts)
        for k, v in right_counts.items():
            total_counts[k] += v

        # Calculate entropy before split
        total_entropy = self.calculate_entropy(total_counts)

        # Entropy for left and right child nodes
        total_left = sum(left_counts.values())
        total_right = sum(right_counts.values())
        total = total_left + total_right

        entropy_left = self.calculate_entropy(left_counts) if total_left > 0 else 0
        entropy_right = self.calculate_entropy(right_counts) if total_right > 0 else 0

        # Weighted average of entropies
        entropy_after_split = (total_left / total) * entropy_left + (total_right / total) * entropy_right

        # Information Gain
        return total_entropy - entropy_after_split

    def attempt_split(self, node):
        best_feature, best_threshold = None, None
        best_info_gain = -float('inf')  # Set to negative infinity to find maximum information gain

        # Loop over features
        for feature in range(len(node.data_points[0][0])):  # Number of features
            feature_values = [x[feature] for x, _ in node.data_points]
            thresholds = np.unique(feature_values)

            for threshold in thresholds:
                left_counts = defaultdict(int)
                right_counts = defaultdict(int)

                for x, y in node.data_points:
                    if x[feature] < threshold:
                        left_counts[y] += 1
                    else:
                        right_counts[y] += 1

                # Calculate information gain
                info_gain = self.calculate_information_gain(left_counts, right_counts)
                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_feature = feature
                    best_threshold = threshold

        # Perform split if a suitable feature and threshold were found
        if best_feature is not None:
            node.feature = best_feature
            node.threshold = best_threshold
            node.is_leaf = False
            node.left = HoeffdingTreeNode()
            node.right = HoeffdingTreeNode()

            # Distribute data points to child nodes
            for x, y in node.data_points:
                if x[best_feature] < best_threshold:
                    node.left.class_counts[y] += 1
                    node.left.data_points.append((x, y))
                else:
                    node.right.class_counts[y] += 1
                    node.right.data_points.append((x, y))

            node.data_points = []  # Clear data points after splitting

    def predict(self, x):
        node = self.root
        while not node.is_leaf:
            if x[node.feature] < node.threshold:
                node = node.left
            else:
                node = node.right

        # If node.class_counts is empty, return a default class (e.g., majority class in the dataset)
        if node.class_counts:
            return max(node.class_counts, key=node.class_counts.get)
        else:
            # Default class if no counts in this leaf (returning the most frequent class seen globally)
            return max(self.root.class_counts, key=self.root.class_counts.get)


In [ ]:
# Usage example
tree = HoeffdingTree()
for index, row in df.iterrows():
    x1 = row[:-1].values  # Features
    y1 = row.iloc[-1]     # Label
    tree.update(x1, y1)

# Predict on new data
y_pred = []
for index, row in X_test.iterrows():
    x1 = row.values  # Features
    prediction = tree.predict(x1)
    y_pred.append(prediction)


In [ ]:
y_act = y_test.values.flatten()

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

# Assuming y_pred and y_act are your predicted and actual labels
accuracy = accuracy_score(y_act, y_pred)
f1 = f1_score(y_act, y_pred, average='macro')  # Use 'micro', 'macro', or 'weighted' based on your case
precision = precision_score(y_act, y_pred, average='macro')
recall = recall_score(y_act, y_pred, average='macro')
cm = confusion_matrix(y_act, y_pred)

# Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score (Macro): {f1:.4f}")
print(f"Precision (Macro): {precision:.4f}")
print(f"Recall (Macro): {recall:.4f}")
print(f"Confusion Matrix:\n{cm}")



Accuracy: 0.5830
F1 Score (Macro): 0.3632
Precision (Macro): 0.5606
Recall (Macro): 0.4015
Confusion Matrix:
[[4575  288   70    3]
 [ 646  902    4   10]
 [2136  131  202    9]
 [ 331  439    7   16]]


# Hoeffding Tree with Fair Information Gain

In [ ]:
import numpy as np
from collections import defaultdict

class FairHoeffdingTreeNode:
    def __init__(self):
        self.is_leaf = True
        self.feature = None
        self.threshold = None
        self.left = None
        self.right = None
        self.class_counts = defaultdict(int)
        self.data_points = []
        self.sensitive_counts = defaultdict(lambda: defaultdict(int))  # Counts based on sensitive attribute

class FairHoeffdingTree:
    def __init__(self, delta=0.01):
        self.root = FairHoeffdingTreeNode()
        self.delta = delta
        self.n = 0  # Number of samples seen

    def hoeffding_bound(self, R, n):
        return np.sqrt((R**2 * np.log(1 / self.delta)) / (2 * n))

    def update(self, x, y, sensitive_value):
        self.n += 1
        node = self.root
        while not node.is_leaf:
            if x[node.feature] < node.threshold:
                node = node.left
            else:
                node = node.right

        node.class_counts[y] += 1
        node.sensitive_counts[sensitive_value][y] += 1
        node.data_points.append((x, y, sensitive_value))

        if self.n % 100 == 0:  # Check for split every 100 samples
            self.attempt_split(node)

    def calculate_entropy(self, class_counts):
        total = sum(class_counts.values())
        return -sum((count / total) * np.log2(count / total) for count in class_counts.values() if count > 0)

    def calculate_information_gain(self, left_counts, right_counts):
        total_counts = defaultdict(int, left_counts)
        for k, v in right_counts.items():
            total_counts[k] += v

        total_entropy = self.calculate_entropy(total_counts)
        total_left = sum(left_counts.values())
        total_right = sum(right_counts.values())
        total = total_left + total_right

        entropy_left = self.calculate_entropy(left_counts) if total_left > 0 else 0
        entropy_right = self.calculate_entropy(right_counts) if total_right > 0 else 0

        entropy_after_split = (total_left / total) * entropy_left + (total_right / total) * entropy_right
        return total_entropy - entropy_after_split

    def calculate_fairness_gain(self, left_sensitive_counts, right_sensitive_counts):
        def discrimination_score(sensitive_counts):
            # Retrieve counts with default 0 if key doesn't exist
            DR = sensitive_counts['sensitive'].get(0, 0)  # Deprived-Rejected
            DG = sensitive_counts['sensitive'].get(1, 0)  # Deprived-Granted
            FR = sensitive_counts['non-sensitive'].get(0, 0)  # Favored-Rejected
            FG = sensitive_counts['non-sensitive'].get(1, 0)  # Favored-Granted

            # Handle division by zero in case of no samples
            if (FG + FR) == 0 or (DG + DR) == 0:
                return 0
            return abs(FG / (FG + FR) - DG / (DG + DR))

        # Calculate discrimination scores before and after the split
        before_split = discrimination_score(left_sensitive_counts) + discrimination_score(right_sensitive_counts)
        after_split = sum(
            len(group) / self.n * discrimination_score(group)
            for group in [left_sensitive_counts, right_sensitive_counts]
        )

        return before_split - after_split

    def calculate_fair_information_gain(self, left_counts, right_counts, left_sensitive_counts, right_sensitive_counts):
        IG = self.calculate_information_gain(left_counts, right_counts)
        FG = self.calculate_fairness_gain(left_sensitive_counts, right_sensitive_counts)
        return IG * FG if FG != 0 else IG

    def attempt_split(self, node):
        best_feature, best_threshold = None, None
        best_fig = -float('inf')

        for feature in range(len(node.data_points[0][0])):  # Number of features
            feature_values = [x[feature] for x, _, _ in node.data_points]
            thresholds = np.unique(feature_values)

            for threshold in thresholds:
                left_counts, right_counts = defaultdict(int), defaultdict(int)
                left_sensitive_counts = defaultdict(lambda: defaultdict(int))
                right_sensitive_counts = defaultdict(lambda: defaultdict(int))

                for x, y, sensitive_value in node.data_points:
                    if x[feature] < threshold:
                        left_counts[y] += 1
                        left_sensitive_counts[sensitive_value][y] += 1
                    else:
                        right_counts[y] += 1
                        right_sensitive_counts[sensitive_value][y] += 1

                fig = self.calculate_fair_information_gain(
                    left_counts, right_counts, left_sensitive_counts, right_sensitive_counts
                )

                if fig > best_fig:
                    best_fig = fig
                    best_feature = feature
                    best_threshold = threshold

        if best_feature is not None:
            node.feature = best_feature
            node.threshold = best_threshold
            node.is_leaf = False
            node.left = FairHoeffdingTreeNode()
            node.right = FairHoeffdingTreeNode()

            for x, y, sensitive_value in node.data_points:
                if x[best_feature] < best_threshold:
                    node.left.class_counts[y] += 1
                    node.left.sensitive_counts[sensitive_value][y] += 1
                    node.left.data_points.append((x, y, sensitive_value))
                else:
                    node.right.class_counts[y] += 1
                    node.right.sensitive_counts[sensitive_value][y] += 1
                    node.right.data_points.append((x, y, sensitive_value))

            node.data_points = []

    def predict(self, x):
        node = self.root
        while not node.is_leaf:
            if x[node.feature] < node.threshold:
                node = node.left
            else:
                node = node.right

        if node.class_counts:
            return max(node.class_counts, key=node.class_counts.get)
        else:
            return max(self.root.class_counts, key=self.root.class_counts.get)


In [ ]:
fair_tree = FairHoeffdingTree(delta=0.01)

# Loop through each row in the dataset and update the tree
for index, row in df.iterrows():
    features = row.drop(['income', 'sex']).values  # Drop target and sensitive columns to get features
    target = row['income']  # The target label
    sensitive_value = row['sex']  # Sensitive attribute value

    # Update the tree with each instance
    fair_tree.update(features, target, sensitive_value)


In [ ]:
y_pred = []
# print(X_test.columns)
for index, row in X_test.iterrows():

    x1 = row.drop(['sex']).values  # Features
    prediction = fair_tree.predict(x1)
    y_pred.append(prediction)
    # print(prediction)


In [ ]:
y_act = y_test.values.flatten()

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

# Assuming y_pred and y_act are your predicted and actual labels
accuracy = accuracy_score(y_act, y_pred)
f1 = f1_score(y_act, y_pred, average='macro')  # Use 'micro', 'macro', or 'weighted' based on your case
precision = precision_score(y_act, y_pred, average='macro')
recall = recall_score(y_act, y_pred, average='macro')
cm = confusion_matrix(y_act, y_pred)

# Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score (Macro): {f1:.4f}")
print(f"Precision (Macro): {precision:.4f}")
print(f"Recall (Macro): {recall:.4f}")
print(f"Confusion Matrix:\n{cm}")



Accuracy: 0.5835
F1 Score (Macro): 0.3637
Precision (Macro): 0.5609
Recall (Macro): 0.4023
Confusion Matrix:
[[4575  288   70    3]
 [ 641  907    4   10]
 [2136  131  202    9]
 [ 328  442    7   16]]
